In [1]:
pip install tensorflow-gpu==2.8.0

Note: you may need to restart the kernel to use updated packages.


In [2]:
pip install -U pip

Note: you may need to restart the kernel to use updated packages.


In [3]:
pip install transformers

Note: you may need to restart the kernel to use updated packages.


In [4]:
pip install datasets

Note: you may need to restart the kernel to use updated packages.


In [5]:
pip install protobuf==3.20

Note: you may need to restart the kernel to use updated packages.


In [6]:
pip install transformers

Note: you may need to restart the kernel to use updated packages.


In [7]:
pip install tqdm

Note: you may need to restart the kernel to use updated packages.


In [8]:
import warnings
# 경고메세지 끄기
warnings.filterwarnings (action='ignore')

In [9]:
import pandas as pd
df = pd.read_csv('./3rd_Data Preprocessing.csv')

In [10]:
df.head()

,labels,name,theme,동영상 제목,댓글,댓글 작성일,PRO
0,,메이플,BM,"환산10만 루미 장비 및 보스 레이드 [메이플스토리, 뚝이]",환산10만할라면 현질얼마해야되나요,2023-10-07T06:58:56Z,환산10만할라면 현질얼마해야되나요
1,,메이플,BM,환불사태 전에는 진짜 게임이 게임이 아니었네;;; [메이플스토리],11:00 채팅창 보니까 쉘로 넘겼다는 말 하나도 없네;;;,2023-06-20T13:11:19Z,채팅창 보니까 쉘로 넘겼다는 말 하나도 없네
2,,메이플,BM,환불사태 전에는 진짜 게임이 게임이 아니었네;;; [메이플스토리],영상제목 ㄹㅇ 웃음벨이네 ㅋㅋㅋㅋㅋㅋㅋ,2023-03-18T01:42:56Z,영상제목 ㄹㅇ 웃음벨이네 ㅋㅋㅋㅋㅋㅋㅋ
3,,메이플,BM,환불사태 전에는 진짜 게임이 게임이 아니었네;;; [메이플스토리],그리고 2년지나고 이재야 생겨난 큐브 천장,2023-04-14T10:01:43Z,그리고 2년지나고 이재야 생겨난 큐브 천장
4,,메이플,BM,환불사태 전에는 진짜 게임이 게임이 아니었네;;; [메이플스토리],근데 청묘님 진짜 메이플 어떻게 시작하신거지 ㄷㄷ,2023-03-22T19:55:52Z,근데 청묘님 진짜 메이플 어떻게 시작하신거지 ㄷㄷ


In [11]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 61220 entries, 0 to 61219
Data columns (total 7 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   labels  61218 non-null  object
 1   name    61220 non-null  object
 2   theme   61220 non-null  object
 3   동영상 제목  61220 non-null  object
 4   댓글      61220 non-null  object
 5   댓글 작성일  61220 non-null  object
 6   PRO     61220 non-null  object
dtypes: object(7)
memory usage: 3.3+ MB


## 긍정,부정 분류 모델 적용

In [12]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

import tensorflow as tf
from tensorflow.keras.losses import SparseCategoricalCrossentropy
from tensorflow.keras.callbacks import EarlyStopping

import torch
from torch.nn import functional as F
from transformers import AutoTokenizer, TFAutoModelForSequenceClassification
from transformers import TFBertForSequenceClassification, BertTokenizer

from sklearn.model_selection import train_test_split
from tensorflow.keras.callbacks import EarlyStopping

import time
import re
import os
import urllib.request
from tqdm import tqdm

In [13]:
# GPU 사용 여부 확인
if tf.config.experimental.list_physical_devices('GPU'):
    device = "cuda"
else:
    device = "cpu"

print(f"사용하고 있는 건? → {device}")

사용하고 있는 건? → cuda


2023-11-28 14:16:06.827054: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:936] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2023-11-28 14:16:06.839170: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:936] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2023-11-28 14:16:06.842121: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:936] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero


In [14]:
# 모델 불러오기
tokenizer = BertTokenizer.from_pretrained("klue/bert-base")
model = TFBertForSequenceClassification.from_pretrained("klue/bert-base", from_pt=True)
model.compile(optimizer=tf.keras.optimizers.Adam(), loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True), metrics=["accuracy"])

# 가중치 저장
model.save_weights("./best_model_gpt(1).h5")

# 가중치 로드
model.load_weights("./best_model_gpt(1).h5")

2023-11-28 14:16:07.463848: I tensorflow/core/platform/cpu_feature_guard.cc:151] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 AVX512F FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-11-28 14:16:07.464459: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:936] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2023-11-28 14:16:07.466198: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:936] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2023-11-28 14:16:07.467667: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:936] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA 

In [15]:
from transformers import pipeline
from tqdm import tqdm

# 트랜스포머 파이프라인 정의
pipe = pipeline("text-classification", model=model, tokenizer=tokenizer, device=0 if device == 'cuda' else -1, max_length=512, function_to_apply='softmax')
label_dict = {'LABEL_0': '중립', 'LABEL_1': '긍정', 'LABEL_2': '부정'}

def prediction(texts):
    # 텍스트에 대한 예측 수행
    results = pipe(texts, truncation=True)  # truncation=True로 설정

    # 예측된 라벨을 라벨 딕셔너리를 사용하여 변환
    predicted_labels = [label_dict[result['label']] for result in results]

    return predicted_labels

# 댓글을 500개씩 split
df_split = np.array_split(df["PRO"], len(df) // 500 + 1)

# 예측 결과를 저장할 리스트
predictions_list = []

# tqdm을 사용하여 진행 상황 표시
for i, df_split_item in enumerate(tqdm(df_split, desc="Processing Batches", unit="batch")):
    predictions_list.append(prediction(df_split_item.tolist()))  # df_split_item을 리스트로 변환하여 전달

# 예측 결과 합치기
predictions_df = np.concatenate(predictions_list)

# 예측 결과를 데이터프레임에 추가
df["예측 감정_gpt"] = predictions_df

# 결과 확인
df["예측 감정_gpt"]

Processing Batches: 100%|██████████| 123/123 [1:51:41<00:00, 54.48s/batch]


0        긍정
1        중립
2        중립
3        중립
4        중립
         ..
61215    긍정
61216    중립
61217    긍정
61218    중립
61219    긍정
Name: 예측 감정_gpt, Length: 61220, dtype: object

In [16]:
np.unique(predictions_df)

array(['긍정', '중립'], dtype='<U2')

In [17]:
df["예측 감정_gpt"].value_counts()

중립    39327
긍정    21893
Name: 예측 감정_gpt, dtype: int64

In [18]:
df

,labels,name,theme,동영상 제목,댓글,댓글 작성일,PRO,예측 감정_gpt
0,,메이플,BM,"환산10만 루미 장비 및 보스 레이드 [메이플스토리, 뚝이]",환산10만할라면 현질얼마해야되나요,2023-10-07T06:58:56Z,환산10만할라면 현질얼마해야되나요,긍정
1,,메이플,BM,환불사태 전에는 진짜 게임이 게임이 아니었네;;; [메이플스토리],11:00 채팅창 보니까 쉘로 넘겼다는 말 하나도 없네;;;,2023-06-20T13:11:19Z,채팅창 보니까 쉘로 넘겼다는 말 하나도 없네,중립
2,,메이플,BM,환불사태 전에는 진짜 게임이 게임이 아니었네;;; [메이플스토리],영상제목 ㄹㅇ 웃음벨이네 ㅋㅋㅋㅋㅋㅋㅋ,2023-03-18T01:42:56Z,영상제목 ㄹㅇ 웃음벨이네 ㅋㅋㅋㅋㅋㅋㅋ,중립
3,,메이플,BM,환불사태 전에는 진짜 게임이 게임이 아니었네;;; [메이플스토리],그리고 2년지나고 이재야 생겨난 큐브 천장,2023-04-14T10:01:43Z,그리고 2년지나고 이재야 생겨난 큐브 천장,중립
4,,메이플,BM,환불사태 전에는 진짜 게임이 게임이 아니었네;;; [메이플스토리],근데 청묘님 진짜 메이플 어떻게 시작하신거지 ㄷㄷ,2023-03-22T19:55:52Z,근데 청묘님 진짜 메이플 어떻게 시작하신거지 ㄷㄷ,중립
...,...,...,...,...,...,...,...,...
61215,,리니지,운영,""" 2023 신버전 밸런스 패치 및 몽환의섬 업데이트 "" - [ 2023 리니지 L...",잼있겠다.. ㅠㅠ 나도 하고싶다,2023-06-30T03:48:27Z,잼있겠다 나도 하고싶다,긍정
61216,,리니지,운영,""" 2023 신버전 밸런스 패치 및 몽환의섬 업데이트 "" - [ 2023 리니지 L...",저녀석 잡는거 다시보여주세용ㅋㅋ 오랫만에보니재밋넹,2023-06-01T14:26:42Z,저녀석 잡는거 다시보여주세용ㅋㅋ 오랫만에보니재밋넹,중립
61217,,리니지,운영,""" 2023 신버전 밸런스 패치 및 몽환의섬 업데이트 "" - [ 2023 리니지 L...",저도 요정 하고있는데 스톰샷 마법이 클릭하면 안배워지고 자꾸 땅에 떨어지는데 어떻...,2023-07-08T13:04:39Z,저도 요정 하고있는데 스톰샷 마법이 클릭하면 안배워지고 자꾸 땅에 떨어지는데 어떻...,긍정
61218,,리니지,운영,""" 2023 신버전 밸런스 패치 및 몽환의섬 업데이트 "" - [ 2023 리니지 L...",형 가디언 잡는거 보여줘...,2023-07-14T09:14:21Z,형 가디언 잡는거 보여줘,중립


In [19]:
df.to_csv('GPT4 예측 결과_231128.csv', index=False, encoding='utf-8')